In [41]:
import os
import numpy as np
import pandas as pd
import math
import csv

import transformers
from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import STSDataReader, TripletReader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryEmbeddingSimilarityEvaluator
from sentence_transformers.readers.InputExample import InputExample

from torch.utils.data import DataLoader

from scipy.spatial.distance import cdist


In [14]:
TRAIN_SPLITS_DATA_DIR = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_data/splitted'

In [2]:
# increase swap size:
# https://superuser.com/questions/1024064/change-swap-file-size-fedora-23

In [3]:
model_wiki = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens')

In [4]:
model_1 = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

In [5]:
model_1.get_max_seq_length()

128

In [6]:
sentences = ['A fox lives in a zoo together with dogs.',
            'Sentences are passed as a list of string.', 
            'The quick brown fox jumps over the lazy dog.']


In [7]:
embeddings = model_wiki.encode(sentences)
sims = cdist(embeddings[0].reshape(-1,1).T, embeddings[1:], "cosine")[0]
sims

array([0.01991861, 0.01082202])

In [8]:
embeddings = model_1.encode(sentences)
sims = cdist(embeddings[0].reshape(-1,1).T, embeddings[1:], "cosine")[0]
sims

array([0.95112492, 0.58440415])

In [9]:
############

In [16]:
#datareader for the regression/raw data
class MyDataReader(STSDataReader):
    '''
    Need to reimplement get_examples method from class STSDataReader because
    our csv file has a header.
    '''
    
    def __init__(self, dataset_folder, **kwargs):
        super(MyDataReader,self).__init__(dataset_folder, **kwargs)
        
    def get_examples(self, filename, header=True, max_examples=0):
        """
        filename specified which data split to use (train.csv, dev.csv, test.csv).
        """
        data = csv.reader(open(os.path.join(self.dataset_folder, filename), encoding="utf-8"),
                          delimiter=self.delimiter, quoting=self.quoting)
        if header:
            next(data, None)  # skip the header
        examples = []
        for id, row in enumerate(data):
            score = float(row[self.score_col_idx])
            if self.normalize_scores:  # Normalize to a 0...1 value
                score = (score - self.min_score) / (self.max_score - self.min_score)

            s1 = row[self.s1_col_idx]
            s2 = row[self.s2_col_idx]
            examples.append(InputExample(guid=filename+str(id), texts=[s1, s2], label=score))

            if max_examples > 0 and len(examples) >= max_examples:
                break

        return examples


In [17]:
myreader_regression = MyDataReader(
                       os.path.join(TRAIN_SPLITS_DATA_DIR, 'queries3_sentences_regression'),
                       s1_col_idx=1,
                       s2_col_idx=3,
                       score_col_idx=2,
                       delimiter=",",
                       quoting=csv.QUOTE_MINIMAL,
                       normalize_scores=False, min_score=0, max_score=1)

myreader_regression.get_examples('queries3_sentences_regression_dev.csv', max_examples=2)

In [28]:
train_data = myreader_regression.get_examples('queries3_sentences_regression_train.csv', max_examples=100000)
train_dataset = SentencesDataset(train_data, show_progress_bar=True, model=model_1)



Convert dataset:  33%|███▎      | 33232/100000 [00:11<00:22, 2922.03it/s]


Convert dataset:  67%|██████▋   | 66867/100000 [00:22<00:11, 2917.87it/s]


Convert dataset:  99%|█████████▊| 98662/100000 [00:33<00:00, 2795.37it/s]


Convert dataset: 100%|██████████| 100000/100000 [00:33<00:00, 2943.77it/s][A


In [29]:
dev_dataset = SentencesDataset(
    myreader_regression.get_examples('queries3_sentences_regression_dev.csv', max_examples=100000), 
    show_progress_bar=True, model=model_1)


Convert dataset: 100%|██████████| 16259/16259 [00:05<00:00, 2909.69it/s]


In [30]:
"""
myreader_rawdata = MyDataReader('/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/',
                       s1_col_idx=1,
                       s2_col_idx=4,
                       score_col_idx=2,
                       delimiter=",",
                       quoting=csv.QUOTE_MINIMAL,
                       normalize_scores=False, min_score=0, max_score=1)

myreader_rawdata.get_examples('queries_od.csv', max_examples=2)
"""

'\nmyreader_rawdata = MyDataReader(\'/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/\',\n                       s1_col_idx=1,\n                       s2_col_idx=4,\n                       score_col_idx=2,\n                       delimiter=",",\n                       quoting=csv.QUOTE_MINIMAL,\n                       normalize_scores=False, min_score=0, max_score=1)\n\nmyreader_rawdata.get_examples(\'queries_od.csv\', max_examples=2)\n'

In [31]:
"""
my_data = SentencesDataset(examples=myreader.get_examples("queries_od.csv", max_examples=100), 
                           model=model_1,
                          show_progress_bar=True)
"""


'\nmy_data = SentencesDataset(examples=myreader.get_examples("queries_od.csv", max_examples=100), \n                           model=model_1,\n                          show_progress_bar=True)\n'

In [ ]:
"""
my_data_l = SentenceLabelDataset(examples=myreader.get_examples("queries_od.csv", max_examples=100), 
                           model=model_wiki,
                          show_progress_bar=True)
"""

In [33]:
num_epochs = 1
train_batch_size = 8

warmup_steps = math.ceil(len(train_dataset)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up

optimizer_class = transformers.AdamW
optimizer_params = {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}

train_loss = losses.CosineSimilarityLoss(model=model_1)

In [38]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

In [39]:
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)

In [42]:
evaluator = BinaryEmbeddingSimilarityEvaluator(dev_dataloader)

In [ ]:
model_1.evaluate(evaluator)







Evaluating:   0%|          | 0/2033 [00:00<?, ?it/s]




Evaluating:   0%|          | 1/2033 [00:03<2:00:20,  3.55s/it]




Evaluating:   0%|          | 2/2033 [00:09<2:22:58,  4.22s/it]




Evaluating:   0%|          | 3/2033 [00:12<2:08:42,  3.80s/it]




Evaluating:   0%|          | 4/2033 [00:14<1:55:16,  3.41s/it]




Evaluating:   0%|          | 5/2033 [00:18<1:56:23,  3.44s/it]




Evaluating:   0%|          | 6/2033 [00:23<2:15:15,  4.00s/it]




Evaluating:   0%|          | 7/2033 [00:27<2:19:57,  4.15s/it]




Evaluating:   0%|          | 8/2033 [00:31<2:09:53,  3.85s/it]




Evaluating:   0%|          | 9/2033 [00:39<2:52:44,  5.12s/it]

In [43]:
MODEL_DIR = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/models'

In [44]:
my_output_path = os.path.join(MODEL_DIR, 'test_model_4')

In [47]:
model_1.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=BinaryEmbeddingSimilarityEvaluator,
          epochs=num_epochs,
          steps_per_epoch=100,
          warmup_steps=warmup_steps,
          optimizer_class=optimizer_class,
          optimizer_params=optimizer_params,
          output_path=my_output_path) # works only when you have an evaluator




Epoch:   0%|          | 0/1 [00:00<?, ?it/s]



Iteration:   0%|          | 0/100 [00:00<?, ?it/s]



Iteration:   1%|          | 1/100 [00:14<24:14, 14.69s/it]



Iteration:   2%|▏         | 2/100 [00:46<25:22, 15.54s/it]



Iteration:   3%|▎         | 3/100 [01:00<25:01, 15.48s/it]



Iteration:   4%|▍         | 4/100 [01:16<24:49, 15.52s/it]



Iteration:   5%|▌         | 5/100 [01:30<24:22, 15.40s/it]



Iteration:   6%|▌         | 6/100 [01:54<24:51, 15.87s/it]



Iteration:   7%|▋         | 7/100 [02:08<24:24, 15.75s/it]



Iteration:   8%|▊         | 8/100 [02:24<24:09, 15.76s/it]



Iteration:   9%|▉         | 9/100 [02:37<23:44, 15.65s/it]



Iteration:  10%|█         | 10/100 [02:51<23:21, 15.57s/it]



Iteration:  11%|█         | 11/100 [03:15<23:41, 15.98s/it]



Iteration:  12%|█▏        | 12/100 [03:34<23:36, 16.09s/it]



Iteration:  13%|█▎        | 13/100 [03:50<23:22, 16.12s/it]



Iteration:  14%|█▍        | 14/100 [04:07<23:08, 16.14s/it]



Iteration:  15%|█▌    

KeyboardInterrupt: 

In [ ]:
model_1.evaluate(evaluator)


In [17]:
#model_1.save(my_output_path)

In [18]:
# load model
# model_1 = SentenceTransformer(my_output_path)

In [19]:
embeddings = model_1.encode(sentences)
sims = cdist(embeddings[0].reshape(-1,1).T, embeddings[1:], "cosine")[0]
sims

array([0.27484048, 0.14570002])